### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the funtion on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [2]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat)) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):
        
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
            
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [3]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10) # use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 36.374362
2 		 10.141005
3 		 3.215756
4 		 2.626877
5 		 2.543185
6 		 2.490835
7 		 2.439490
8 		 2.382697
9 		 2.318612
10 		 2.246726


In [4]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.06608662  8.77637425 10.69843618 10.14812931]
 [ 8.77188695  7.48345968  9.17680749  8.56401018]
 [ 8.00729306  6.51789896  8.66112644  7.60242582]
 [ 9.49688651  8.43682248  9.93647379  9.59506465]
 [ 8.68533805  6.90088147  9.4919622   8.2847629 ]
 [ 6.512174    5.78315435  7.74345399  6.68155248]
 [ 9.48685528  7.76141777  9.79178937  9.08133035]
 [ 8.3118951   6.99625547  8.78633434  8.08177093]
 [ 8.40826089  7.50977468  8.87386324  8.48757276]
 [ 7.94886986  6.25194158  8.39571137  7.48487629]
 [ 8.78053589  7.76574155  9.37592523  8.88598909]
 [ 9.39340704  8.57805985 10.00287513  9.65118405]
 [ 9.15816388  8.31069798  9.53714202  9.40118318]
 [ 6.56911359  6.10864005  6.50006158  6.86543408]
 [ 9.79605021  8.9153298   9.60742428 10.02065651]
 [ 9.89894295  8.54570578 10.11016559  9.72041758]
 [ 8.34687508  7.34020423  8.4970761   8.39340549]
 [ 7.25578559  6.93255315  6.37095571  7.51332137]
 [ 8.09664803  7.00012239  9.29504644  8.24239744]
 [ 8.14946537  7.02022773  8.62

**Write your summary here.**

**The predicted ratings from the dot product are already starting to look a lot like the original data values even after only 10 iterations.  You can see some extreme low values that are not captured well yet.  The 5 in the second to last row in the first column is predicted as an 8, and the 4 in the second row and second column is predicted to be a 7.  Clearly the model is not done learning, but things are looking good.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [5]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 45.000693
2 		 17.056829
3 		 4.669893
4 		 2.807676
5 		 2.622890
6 		 2.578666
7 		 2.547615
8 		 2.515921
9 		 2.481100
10 		 2.442357
11 		 2.399144
12 		 2.350916
13 		 2.297108
14 		 2.237153
15 		 2.170512
16 		 2.096717
17 		 2.015425
18 		 1.926487
19 		 1.830020
20 		 1.726482
21 		 1.616735
22 		 1.502076
23 		 1.384213
24 		 1.265188
25 		 1.147230
26 		 1.032563
27 		 0.923201
28 		 0.820762
29 		 0.726351
30 		 0.640529
31 		 0.563358
32 		 0.494508
33 		 0.433390
34 		 0.379287
35 		 0.331454
36 		 0.289186
37 		 0.251852
38 		 0.218904
39 		 0.189871
40 		 0.164345
41 		 0.141966
42 		 0.122412
43 		 0.105387
44 		 0.090617
45 		 0.077848
46 		 0.066843
47 		 0.057384
48 		 0.049273
49 		 0.042330
50 		 0.036394
51 		 0.031323
52 		 0.026992
53 		 0.023293
54 		 0.020130
55 		 0.017425
56 		 0.015108
57 		 0.013121
58 		 0.011414
59 		 0.009945
60 		 0.008679
61 		 0.007585
62 		 0.006639
63 		 0.005820
64 		

In [6]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.99999923  9.99999954  9.99999972 10.00000092]
 [10.00000297  4.00000172  9.00000104  9.99999651]
 [ 8.00000158  9.00000084 10.0000005   4.9999982 ]
 [ 9.00000225  8.00000118 10.00000069  9.9999974 ]
 [ 9.9999991   4.99999936  8.99999958  9.00000107]
 [ 5.99999965  3.99999972  9.99999981  6.00000041]
 [ 8.99999603  7.99999768  9.99999858  9.00000463]
 [ 9.99999867  4.99999926  8.99999955  8.00000153]
 [ 7.00000033  8.00000016 10.00000012  7.99999966]
 [ 8.9999997   4.99999984  8.99999992  7.00000035]
 [ 8.99999739  7.99999853  9.99999913  8.00000306]
 [ 8.99999909  9.99999956  9.99999976  9.00000107]
 [ 9.99999857  8.9999993   9.99999961  8.00000166]
 [ 5.00000127  8.00000074  5.00000048  7.99999856]
 [10.00000272  8.00000166 10.00000102  9.99999681]
 [ 9.00000252  9.00000144 10.00000087  9.99999708]
 [ 8.99999753  7.99999857  7.99999913  8.0000029 ]
 [ 9.99999967  7.99999979  0.99999988 10.0000004 ]
 [ 4.99999755  5.99999863  9.99999917 10.00000283]
 [ 8.0000038   7.00000218 10.00

**Write your summary here.**

**In this case, we were able to completely reconstruct the item-movie matrix to obtain an essentially 0 mean squared error. I obtained 0 MSE on iteration 165.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [7]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [8]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 38.115026
2 		 12.249133
3 		 3.940480
4 		 2.886955
5 		 2.719808
6 		 2.658752
7 		 2.616912
8 		 2.575852
9 		 2.530422
10 		 2.478694
11 		 2.419595
12 		 2.352386
13 		 2.276604
14 		 2.192119
15 		 2.099223
16 		 1.998699
17 		 1.891860
18 		 1.780509
19 		 1.666820
20 		 1.553141
21 		 1.441751
22 		 1.334613
23 		 1.233191
24 		 1.138357
25 		 1.050419
26 		 0.969224
27 		 0.894326
28 		 0.825140
29 		 0.761069
30 		 0.701585
31 		 0.646256
32 		 0.594748
33 		 0.546794
34 		 0.502173
35 		 0.460681
36 		 0.422122
37 		 0.386294
38 		 0.352999
39 		 0.322043
40 		 0.293248
41 		 0.266452
42 		 0.241519
43 		 0.218335
44 		 0.196809
45 		 0.176867
46 		 0.158446
47 		 0.141494
48 		 0.125959
49 		 0.111787
50 		 0.098923
51 		 0.087303
52 		 0.076860
53 		 0.067519
54 		 0.059202
55 		 0.051829
56 		 0.045318
57 		 0.039589
58 		 0.034563
59 		 0.030164
60 		 0.026324
61 		 0.022976
62 		 0.020062
63 		 0.017527
64 		

In [9]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 9.222657513386231:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [10]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20) #fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 23.244157
2 		 10.718166
3 		 7.346297
4 		 5.672509
5 		 4.617008
6 		 3.876526
7 		 3.324689
8 		 2.897158
9 		 2.556792
10 		 2.280343
11 		 2.052382
12 		 1.862188
13 		 1.702011
14 		 1.566066
15 		 1.449914
16 		 1.350067
17 		 1.263732
18 		 1.188648
19 		 1.122964
20 		 1.065168


In [17]:
first_1000_users

matrix([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]])

`6.` Now that you have a set of predictions for each user-movie pair.  Let's answer a few questions about your results. Provide the correct values to each of the variables below, and check your solutions using the tests below.

In [11]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users)) # How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings # How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [12]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
